In [ ]:
import numpy as np

import theano.tensor as TT
import time
import random

import pymc3 as pm
import arviz as az

## Defino las funciones que voy a usar en el esquema bayesiano para ajustar la señal reflejada por el target.

In [ ]:
## solo un medio dieléctrico

def toFit_1L(ep):
    
    S1_real = np.real(Scal*R_ep(ep))
    S1_imag = np.imag(Scal*R_ep(ep))
  
    # return np.asarray([S1_real, S1_imag])
    dim = int(np.shape(Scal)[0])
 
    a = TT.zeros((2,dim))
    a1 = TT.set_subtensor(a[0], S1_real)
    a2 = TT.set_subtensor(a1[1], S1_imag)
 
    return a2
##----------------------------------------------------------------------##

## Función principal del esquema bayesiano

In [ ]:
def modelo(S2, samples, sigmas,n_layers):
    
    '''
    S2: medición sobre el target de interés: tiene tamaño (2,n_frec);
    el 2 es porque separé parte Real e Imaginaria de la señal.
    samples: escalar, nro de muestras para la inferencia bayesiana.
    sigmas: varianza de la medición, de tamaño (2, n_frec).
    
    n_layers = nro de medios materiales a usar en el modelo teórico.
    
    OBS:
        - S1 y S2 son complejos de dimensión N
        - frecs es un vector real de dimensión N
    '''
    
    ## tomo parte real e imaginaria de la señal a ajustar
    S2_r = np.real(S2)
    S2_i = np.imag(S2)
    
    ## defino el número de capas del modelo
    if(n_layers == 1):
        
        ep1r_min, ep1r_max = 4, 30

        ranges = {}
        ranges['ep1_r'] = ep1r_min, ep1r_max
        
        with pm.Model() as model:
            ep1_r = pm.Uniform('ep1_r', ranges['ep1_r'][0], ranges['ep1_r'][1])
            #ep1_i = pm.Uniform('ep1_i', 0.25*ranges['ep1_r'][0], 6*ranges['ep1_r'][0])
            ep1_i = pm.Uniform('ep1_i', 3.0, 10)

        def f(ep1_ = ep1_r + 1j*ep1_i ):
            return toFit_1L(ep1_)

        with model:
            function_pm = pm.Deterministic('S2_teo', f())
            observations = pm.Normal( 'S2_med',  mu = function_pm, sigma = sigmas, observed = np.asarray([S2_r,S2_i]))
            trace = pm.sample_smc(samples, cores=6, parallel=True)
            #ppc = pm.sample_posterior_predictive(trace, var_names=["ep1_r","ep1_i","S2_med"], random_seed=4, samples=10)
            
        return trace#, ppc, model
        
           
    
    else:
        print('N layers incorrecto')